In [4]:
import numpy, math, os.path, shutil, errno

# Set up dummy project

In [5]:
# Create a test project with dummy data
NUM_SUBJECTS = 3
NUM_VISITS = 2

HEIGHT_MEAN = 1700
HEIGHT_STD = 150
WEIGHT_MEAN = 70
WEIGHT_STD = 25
HEAD_CIRC_MEAN = 570
HEAD_CIRC_STD = 30

subjects = ['subject{}'.format(i) for i in range(NUM_SUBJECTS)]
visits = ['visit{}'.format(i) for i in range(NUM_VISITS)]

project_dir = os.path.join(os.environ['HOME'], 'Desktop', 'arcana_tutorial')
# Clean old directory
shutil.rmtree(project_dir, ignore_errors=True)
os.mkdir(project_dir)
for subj in subjects:
    for visit in visits:
        session_dir = os.path.join(project_dir, subj, visit)
        try:
            os.makedirs(session_dir)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
        with open(os.path.join(session_dir, 'metrics.txt'), 'w') as f:
            f.write('height {}\n'.format(numpy.random.randn() * HEIGHT_STD + HEIGHT_MEAN))
            f.write('weight {}\n'.format(numpy.random.randn() * WEIGHT_STD + WEIGHT_MEAN))
            f.write('head_circ {}\n'.format(numpy.random.randn() * HEAD_CIRC_STD + HEAD_CIRC_MEAN))
print("Created project in {} directory".format(project_dir))

Created project in /Users/tclose/Desktop/arcana_tutorial directory


# Create interface for 'grep' tool

In [6]:
from nipype.interfaces.base import (
    TraitedSpec, traits, File, isdefined,
    CommandLineInputSpec, CommandLine)

class GrepInputSpec(CommandLineInputSpec):
    match_str = traits.Str(argstr='-e %s', position=0,
                           desc="The string to search for")
    in_file = File(argstr='%s', position=1,
                   desc="The file to search")
    out_file = File(genfile=True, argstr='> %s', position=2,
                    desc=("The file to contain the search results"))


class GrepOutputSpec(TraitedSpec):
    out_file = File(exists=True, desc="The search results")


class Grep(CommandLine):
    """Creates a zip archive from a given folder"""

    _cmd = 'grep'
    input_spec = GrepInputSpec
    output_spec = GrepOutputSpec

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['out_file'] = self._gen_filename('out_file')
        return outputs

    def _gen_filename(self, name):
        if name == 'out_file':
            if isdefined(self.inputs.out_file):
                fname = self.inputs.out_file
            else:
                fname = os.path.join(os.getcwd(), 'search_results.txt')
        else:
            assert False
        return fname


In [7]:
grep = Grep()
grep.inputs.in_file = os.path.join(project_dir, 'subject0', 'visit0', 'metrics.txt')
grep.inputs.match_str = 'height'
results = grep.run()
print(results.outputs)
print(os.getcwd())
print(os.listdir(os.getcwd()))


out_file = /Users/tclose/Documents/Presentations/2019-02-04-Vector-Arcana-2/search_results.txt

/Users/tclose/Documents/Presentations/2019-02-04-Vector-Arcana-2
['MBI-Vector 2019-02-04.pptx', 'MBI-Vector-2019-02-04.ipynb', '.DS_Store', 'search_results.txt', '~$MBI-Vector 2019-02-04.pptx', 'graph.png', 'test-dir', 'my_workflow', '.ipynb_checkpoints', 'graph.dot', 'pyscript.m']


In [8]:
for subj in subjects:
    for visit in visits:
        grep = Grep()
        grep.inputs.match_str = 'height'
        grep.inputs.in_file = os.path.join(project_dir, subj, visit, 'metrics.txt')
        grep.inputs.out_file = os.path.join(project_dir, subj, visit, 'grep.txt')
        result = grep.run()
        print('Processed {}'.format(result.outputs.out_file))

Processed /Users/tclose/Desktop/arcana_tutorial/subject0/visit0/grep.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject0/visit1/grep.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject1/visit0/grep.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject1/visit1/grep.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject2/visit0/grep.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject2/visit1/grep.txt


# Create interface for 'awk' tool

In [9]:
from nipype.interfaces.base import (
    TraitedSpec, traits, File, isdefined,
    CommandLineInputSpec, CommandLine)

class AwkInputSpec(CommandLineInputSpec):
    format_str = traits.Str(argstr="'%s'", position=0,
                            desc="The string to search for")
    in_file = File(argstr='%s', position=1,
                   desc="The file to parse")
    out_file = File(genfile=True, argstr='> %s', position=2,
                    desc=("The file to contain the parsed results"))


class AwkOutputSpec(TraitedSpec):
    out_file = File(exists=True, desc="The parsed results")


class Awk(CommandLine):
    """Creates a zip archive from a given folder"""

    _cmd = 'awk'
    input_spec = AwkInputSpec
    output_spec = AwkOutputSpec

    def _list_outputs(self):
        outputs = self._outputs().get()
        outputs['out_file'] = self._gen_filename('out_file')
        return outputs

    def _gen_filename(self, name):
        if name == 'out_file':
            if isdefined(self.inputs.out_file):
                fname = self.inputs.out_file
            else:
                fname = os.path.join(os.getcwd(), 'awk_results.txt')
        else:
            assert False
        return fname

In [10]:
for subj in subjects:
    for visit in visits:
        awk = Awk()
        awk.inputs.format_str = '{print $2}'
        awk.inputs.in_file = os.path.join(project_dir, subj, visit, 'grep.txt')
        awk.inputs.out_file = os.path.join(project_dir, subj, visit, 'awk.txt')
        result = awk.run()
        print('Processed {}'.format(result.outputs.out_file))

Processed /Users/tclose/Desktop/arcana_tutorial/subject0/visit0/awk.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject0/visit1/awk.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject1/visit0/awk.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject1/visit1/awk.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject2/visit0/awk.txt
Processed /Users/tclose/Desktop/arcana_tutorial/subject2/visit1/awk.txt


# Example Matlab interface

In [11]:
from nipype.interfaces.base import traits
from nipype.interfaces.base import TraitedSpec
from nipype.interfaces.matlab import MatlabCommand, MatlabInputSpec


class HelloWorldInputSpec(MatlabInputSpec):
    name = traits.Str(mandatory=True,
                      desc='Name of person to say hello to')


class HelloWorldOutputSpec(TraitedSpec):
    matlab_output = traits.Str()


class HelloWorld(MatlabCommand):
    input_spec = HelloWorldInputSpec
    output_spec = HelloWorldOutputSpec

    def _my_script(self):
        """This is where you implement your script"""
        script = """
        disp('Hello {}')
        """.format(self.inputs.name)
        return script

    def run(self, **inputs):
        # Inject your script
        self.inputs.script = self._my_script()
        results = super(MatlabCommand, self).run(**inputs)
        stdout = results.runtime.stdout
        # Attach stdout to outputs to access matlab results
        results.outputs.matlab_output = stdout
        return results

    def _list_outputs(self):
        outputs = self._outputs().get()
        return outputs


In [12]:
hello = HelloWorld()
hello.inputs.name = 'Tom'
out = hello.run()
print(out.outputs.matlab_output)


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (maci64)
                             February 23, 2018

[Warning: Name is nonexistent or not a directory:
/Users/tclose/git/ni/sti/Support_Functions/NII/wavelet_src] 
 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 
Executing pyscript at 04-Feb-2019 13:50:39:
-----------------------------------------------------------------------------------------------------
MATLAB Version: 9.4.0.813654 (R2018a)
MATLAB License Number: 678256
Operating System: Mac OS X  Version: 10.13.6 Build: 17G65 
Java Version: Java 1.8.0_144-b01 with Oracle Corporation Java HotSpot(TM) 64-Bit Server VM mixed mode
-----------------------------------------------------------------------------------------------------
MATLAB                                                Version 9.4         (R2018a

# Utility 'concat' Python interface

In [13]:
from nipype.interfaces.base import (
    TraitedSpec, traits, BaseInterface, File, isdefined, InputMultiPath)

class ConcatFloatsInputSpec(TraitedSpec):
    in_files = InputMultiPath(desc='file name')


class ConcatFloatsOutputSpec(TraitedSpec):
    out_list = traits.List(traits.Float, desc='input floats')


class ConcatFloats(BaseInterface):
    """Joins values from a list of files into a single list"""

    input_spec = ConcatFloatsInputSpec
    output_spec = ConcatFloatsOutputSpec

    def _list_outputs(self):
        out_list = []
        for path in self.inputs.in_files:
            with open(path) as f:
                val = float(f.read())
                out_list.append(val)
        outputs = self._outputs().get()
        outputs['out_list'] = out_list
        return outputs

    def _run_interface(self, runtime):
        # Do nothing
        return runtime

# Python interface using Numpy

In [14]:
from nipype.interfaces.base import (
    TraitedSpec, traits, BaseInterface)

class ExtractMetricsInputSpec(TraitedSpec):
    in_list = traits.List(traits.Float, desc='input floats')


class ExtractMetricsOutputSpec(TraitedSpec):
    std = traits.Float(desc="The standard deviation")
    avg = traits.Float(desc="The average")


class ExtractMetrics(BaseInterface):
    """Joins values from a list of files into a single list"""

    input_spec = ExtractMetricsInputSpec
    output_spec = ExtractMetricsOutputSpec

    def _list_outputs(self):
        values = self.inputs.in_list
        outputs = self._outputs().get()
        outputs['std'] = numpy.std(values)
        outputs['avg'] = numpy.average(values)
        return outputs

    def _run_interface(self, runtime):
        # Do nothing
        return runtime


# Manual run concatenation and metric extraction over project

In [15]:
in_files = []
for subj in subjects:
    for visit in visits:
        in_files.append(os.path.join(project_dir, subj, visit, 'awk.txt'))

concat_floats = ConcatFloats()
concat_floats.inputs.in_files = in_files
result = concat_floats.run()
print('Output list {}'.format(result.outputs.out_list))

extract_metrics = ExtractMetrics()
extract_metrics.inputs.in_list = result.outputs.out_list
result = extract_metrics.run()
print('Average: {}'.format(result.outputs.avg))
print('Std.: {}'.format(result.outputs.std))

Output list [1865.3997988028266, 1654.1872032510767, 1925.8241108822733, 1848.7633659299897, 1727.017752746845, 1777.1652630024328]
Average: 1799.726249102574
Std.: 90.91705203634373


# NiPype workflow for 

In [16]:
from nipype.pipeline import engine as pe  # pypeline engine
from nipype.interfaces.utility import IdentityInterface, Merge
from nipype.interfaces.io import DataGrabber

# Create workflow
workflow = pe.Workflow(name='my_workflow')

# Create subjects iterator
subject_iterator = pe.Node(IdentityInterface(['subject_id']),
                           name='subject_iterator')
workflow.add_nodes([subject_iterator])
subject_iterator.iterables = ('subject_id', subjects)

# Create visits iterator
visit_iterator = pe.Node(IdentityInterface(['visit_id']),
                         name='visit_iterator')
workflow.add_nodes([visit_iterator])
visit_iterator.iterables = ('visit_id', visits)

# Create data grabber
datasource = pe.Node(
    interface=DataGrabber(
        infields=['subject_id', 'visit_id'], outfields=['metrics']),
    name='datasource')
datasource.inputs.template = "%s/%s/metrics.txt"
datasource.inputs.base_directory = project_dir
datasource.inputs.sort_filelist = True
datasource.inputs.template_args = {'metrics': [['subject_id', 'visit_id']]}
workflow.add_nodes([datasource])

# Create grep node
grep = pe.Node(Grep(), name='grep')
grep.inputs.match_str = 'height'
workflow.add_nodes([grep])

# Create awk node
awk = pe.Node(Awk(), name='awk')
awk.inputs.format_str = '{print $2}'
workflow.add_nodes([awk])

# Merge subject and visit iterators
merge_visits = pe.JoinNode(Merge(1), name='merge_visits', joinfield='in1',
                           joinsource='visit_iterator')
merge_subjects = pe.JoinNode(Merge(1), name='merge_subjects', joinfield='in1',
                    joinsource='subject_iterator')
merge_subjects.inputs.ravel_inputs = True
workflow.add_nodes([merge_subjects, merge_visits])
                                            
# Concat floats node
concat = pe.Node(ConcatFloats(), name='concat')
workflow.add_nodes([concat])
                                            
# Extract metrics Node
extract_metrics = pe.Node(ExtractMetrics(), name='extract')
workflow.add_nodes([extract_metrics])
                                            
# Connect Nodes together                          
workflow.connect(subject_iterator, 'subject_id', datasource, 'subject_id')
workflow.connect(visit_iterator, 'visit_id', datasource, 'visit_id')
workflow.connect(datasource, 'metrics', grep, 'in_file')
workflow.connect(grep, 'out_file', awk, 'in_file')
workflow.connect(awk, 'out_file', merge_visits, 'in1')
workflow.connect(merge_visits, 'out', merge_subjects, 'in1')
workflow.connect(merge_subjects, 'out', concat, 'in_files')
workflow.connect(concat, 'out_list', extract_metrics, 'in_list')
             

# Run workflow
workflow.run()



190204-13:50:44,903 nipype.workflow INFO:
	 Workflow my_workflow settings: ['check', 'execution', 'logging', 'monitoring']
190204-13:50:44,933 nipype.workflow INFO:
	 Running serially.
190204-13:50:44,934 nipype.workflow INFO:
	 [Node] Setting-up "my_workflow.datasource" in "/private/tmp/tmpm3cjtm10/my_workflow/_subject_id_subject2/_visit_id_visit1/datasource".
190204-13:50:44,939 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190204-13:50:44,947 nipype.workflow INFO:
	 [Node] Finished "my_workflow.datasource".
190204-13:50:44,948 nipype.workflow INFO:
	 [Node] Setting-up "my_workflow.grep" in "/private/tmp/tmp6jxx_46u/my_workflow/_subject_id_subject2/_visit_id_visit1/grep".
190204-13:50:44,953 nipype.workflow INFO:
	 [Node] Running "grep" ("__main__.Grep"), a CommandLine Interface with command:
grep -e height /Users/tclose/Desktop/arcana_tutorial/subject2/visit1/metrics.txt > /private/tmp/tmp6jxx_46u/my_workflow/_subject_id_subject2/_visit_id_

190204-13:50:45,856 nipype.workflow INFO:
	 [Node] Running "merge_visits" ("nipype.interfaces.utility.base.Merge")
190204-13:50:45,864 nipype.workflow INFO:
	 [Node] Finished "my_workflow.merge_visits".
190204-13:50:45,865 nipype.workflow INFO:
	 [Node] Setting-up "my_workflow.datasource" in "/private/tmp/tmp7ebjpw79/my_workflow/_subject_id_subject0/_visit_id_visit0/datasource".
190204-13:50:45,872 nipype.workflow INFO:
	 [Node] Running "datasource" ("nipype.interfaces.io.DataGrabber")
190204-13:50:45,879 nipype.workflow INFO:
	 [Node] Finished "my_workflow.datasource".
190204-13:50:45,880 nipype.workflow INFO:
	 [Node] Setting-up "my_workflow.grep" in "/private/tmp/tmpa63cx6kx/my_workflow/_subject_id_subject0/_visit_id_visit0/grep".
190204-13:50:45,887 nipype.workflow INFO:
	 [Node] Running "grep" ("__main__.Grep"), a CommandLine Interface with command:
grep -e height /Users/tclose/Desktop/arcana_tutorial/subject0/visit0/metrics.txt > /private/tmp/tmpa63cx6kx/my_workflow/_subject_id_s

In [17]:
workflow.write_graph()

190204-13:50:46,293 nipype.workflow INFO:
	 Generated workflow graph: /Users/tclose/Documents/Presentations/2019-02-04-Vector-Arcana-2/graph.png (graph2use=hierarchical, simple_form=True).


'/Users/tclose/Documents/Presentations/2019-02-04-Vector-Arcana-2/graph.png'

In [18]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('graph.png')
fig = plt.figure()
plt.imshow(img)
fig.set_size_inches(15, 15)
plt.show()

<Figure size 1500x1500 with 1 Axes>

# Example Arcana Study class

Create the Study class, defining its constituent data and option specifications

In [19]:
import arcana
from arcana import (Study, StudyMetaClass, ParameterSpec, AcquiredFilesetSpec,
                    FilesetSpec, FieldSpec)
from arcana.data.file_format.standard import text_format


class MyStudy(Study, metaclass=StudyMetaClass): 
    
    add_data_specs = [
        AcquiredFilesetSpec('body_metrics', text_format),
        FilesetSpec('selected_metric', text_format, 'extract_metrics_pipeline'),
        FieldSpec('average', float, 'statistics_pipeline', frequency='per_study'),
        FieldSpec('std_dev', float, 'statistics_pipeline',
                  frequency='per_study')]
    
    add_param_specs = [
        ParameterSpec('metric_of_interest', 'height')]
    
    def extract_metrics_pipeline(self, **name_maps):
        pipeline = self.new_pipeline(
            name='extract_metrics',
            name_maps=name_maps,
            desc="Extract metrics from file")
        
        grep = pipeline.add(
            'grep',
            Grep(
                match_str=self.parameter('metric_of_interest')),
            inputs={
                'in_file': ('body_metrics', text_format)})
        
        pipeline.add(
            'awk',
            Awk(
                format_str='{print $2}'),
            inputs={
                'in_file': (grep, 'out_file')},
            outputs={
                'selected_metric': ('out_file', text_format)})
        
        return pipeline

    def statistics_pipeline(self, **name_maps):
        pipeline = self.new_pipeline(
            name='statistics',
            name_maps=name_maps,
            desc="Calculate statistics")
        
        merge_visits = pipeline.add(
            'merge_visits',
            Merge(
                numinputs=1),
            inputs={
                'in1': ('selected_metric', text_format)})
        
        merge_subjects = pipeline.add(
            'merge_subjects',
            Merge(
                numinputs=1,
                ravel_inputs=True),
            inputs={
                'in1': (merge_visits, 'out')})
        
        concat = pipeline.add(
            'concat',
            ConcatFloats(),
            inputs={
                'in_files': (merge_subjects, 'out')})
        
        extract_metrics = pipeline.add(
            'extract_metrics', 
            ExtractMetrics(),
            inputs={
                'in_list': (concat, 'out_list')},
            outputs={
                'average': ('avg', float),
                'std_dev': ('std', float)})        
        
        return pipeline

# Applying MyStudy to test data set

In [20]:
import os.path as op
from arcana import (
    FilesetSelector, DirectoryRepository, LinearProcessor, StaticEnvironment)


my_study = MyStudy(
    name='example_mystudy',
    repository=DirectoryRepository(project_dir, depth=2),
    processor=LinearProcessor(work_dir=op.expanduser('~/work')),
    environment=StaticEnvironment(),
    inputs=[FilesetSelector('body_metrics', 'metrics', text_format)],
    parameters={'metric_of_interest': 'height'})

# Applying Banana DwiStudy to example dataset

In [21]:
from banana.study.mri.dwi import DwiStudy

dwi_study = DwiStudy(
    name='example_diffusion',
    repository=DirectoryRepository(
        op.join(op.expanduser('~'), 'Downloads', 'test-dir'), depth=0),
    processor=LinearProcessor(work_dir=op.expanduser('~/work')),
    environment=StaticEnvironment(),
    inputs=[FilesetSelector('magnitude', 'R_L.*', dicom_format, is_regex=True),
            FilesetSelector('reverse_phase', 'L_R.*', dicom_format,
                            is_regex=True)],
    parameters={'num_global_tracks': int(1e6)})


# Generate whole brain tracks and return path to cached dataset
wb_tcks = study.data('global_tracks')
for sess_tcks in wb_tcks:
    print("Performed whole-brain tractography for {}:{} session, the results "
          "are stored at '{}'"
          .format(sess_tcks.subject_id, sess_tcks.visit_id, sess_tcks.path))

/Users/tclose/git/ni/banana/banana/interfaces/custom/dicom.py:13: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  import nibabel.nicom.csareader as csareader
/Users/tclose/git/ni/banana/banana/interfaces/custom/motion_correction.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/local/Cellar/python/3.7.2_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.2_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _

NameError: name 'dicom_format' is not defined